In [14]:
import pandas as pd 
import numpy as np

import seaborn as sn
import confusion_matrix_pretty_print
from confusion_matrix_pretty_print import plot_confusion_matrix_from_data
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix,classification_report,precision_score
import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Softmax, Dropout

In [15]:
tripleData = pd.read_csv('../../triple42702_ShuffledPaired.csv')

In [16]:
fold = 10 
interval = int(42702/fold)
auprListpositive1 = []
aucListpositive1 = []

auprListmines1 = []
aucListmines1 = []

In [ ]:

for split1 in range(10):
    split2 = split1 + 1
    
    trainIndex = list(range(0,split1*interval))
    trainIndex.extend(list(range(split2*interval,42702)))

    if split1==9:
        testIndex = list(range(split1*interval,42702))
    else:
        testIndex = list(range(split1*interval,split2*interval))

    dataTrain = tripleData.iloc[trainIndex,:]
    dataTest = tripleData.iloc[testIndex,:]
    print(min(testIndex),max(testIndex))


    X_train, X_test = dataTrain.values[:,3:], dataTest.values[:,3:]
    y_train, y_test = dataTrain.values[:,2].astype(int), dataTest.values[:,2].astype(int)
    trainNum = len(X_train)
    testNum = len(X_test)

    #reshape data to fit model
    X_train = X_train.reshape(trainNum,16,71,1)
    X_test = X_test.reshape(testNum,16,71,1)

    y_train = y_train + 1
    y_test  = y_test + 1
    y_train = y_train / 2
    y_test  = y_test / 2
    # print(y_train[0], y_test[0])

    from keras.utils import to_categorical
    #one-hot encode target column
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    # y_test[0]
    #create model
    model = Sequential()
    #add model layers
    # kernel_initializer='uniform',
    # kernel_initializer='uniform',
    # kernel_initializer='uniform',
    # kernel_initializer='uniform',
    model.add(Conv2D(128, kernel_size=4, activation='relu', input_shape=(16,71,1)))
    # model.add(Conv2D(64, kernel_size=2, activation='relu'))

    model.add(Conv2D(32, kernel_size=4, activation='relu'))
    # model.add(Conv2D(16, kernel_size=2, activation='relu'))
    model.add(Conv2D(8, kernel_size=4, activation='relu'))
    model.add(Flatten())
    model.add(Dense( 1024, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense( 64, activation='relu'))
    model.add(Dense( 2, activation='relu'))
    model.add(Softmax(128))
#     model.summary()


    #compile model using accuracy to measure model performance
    from keras import optimizers
    from keras import metrics as kmetr


    adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)
    # model.compile(loss='hinge', optimizer=adam, metrics=[kmetr.categorical_accuracy])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) ## Minist


    # # Load the model's saved weights.
    # model.load_weights('cnn.h5')

    #train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)


    # Saveing the Model
    model.save_weights('cnn_'+str(split2)+'.h5')


    #predict first 4 images in the test set
    predit = model.predict(X_test)

    #actual results for first 4 images in test set
    print(predit[:4])


    #from sklearn.metrics import precision_recall_curve, roc_curve
    from sklearn.metrics import auc, precision_recall_curve, roc_curve
    prec, rec, thr = precision_recall_curve(y_test[:,0], predit[:,0])
    aupr_val = auc(rec, prec)
    fpr, tpr, thr = roc_curve(y_test[:,0], predit[:,0])
    auc_val = auc(fpr, tpr)
    aucListmines1.append(auc_val)
    auprListmines1.append(aupr_val)
    print(aupr_val,auc_val)

    #from sklearn.metrics import precision_recall_curve, roc_curve
    from sklearn.metrics import auc, precision_recall_curve, roc_curve
    prec, rec, thr = precision_recall_curve(y_test[:,1], predit[:,1])
    aupr_val = auc(rec, prec)
    fpr, tpr, thr = roc_curve(y_test[:,1], predit[:,1])
    auc_val = auc(fpr, tpr)
    aucListpositive1.append(auc_val)
    auprListpositive1.append(aupr_val)
    print(aupr_val , auc_val )

0 4269
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 13, 68, 128)       2176      
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 10, 65, 32)        65568     
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 7, 62, 8)          4104      
_________________________________________________________________
flatten_14 (Flatten)         (None, 3472)              0         
_________________________________________________________________
dense_40 (Dense)             (None, 1024)              3556352   
_________________________________________________________________
dropout_14 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_41 (Dense)             (None, 64)       

Train on 38432 samples, validate on 4270 samples
Epoch 1/5
38432/38432 [==============================] - 253s 7ms/step - loss: 0.3466 - acc: 0.8418 - val_loss: 0.2635 - val_acc: 0.8817
Epoch 2/5
38432/38432 [==============================] - 235s 6ms/step - loss: 0.2112 - acc: 0.9037 - val_loss: 0.1890 - val_acc: 0.9255
Epoch 3/5
38432/38432 [==============================] - 236s 6ms/step - loss: 0.1275 - acc: 0.9475 - val_loss: 0.1608 - val_acc: 0.9412
Epoch 4/5
38432/38432 [==============================] - 257s 7ms/step - loss: 0.0852 - acc: 0.9653 - val_loss: 0.1531 - val_acc: 0.9450
Epoch 5/5
38432/38432 [==============================] - 225s 6ms/step - loss: 0.0609 - acc: 0.9755 - val_loss: 0.1849 - val_acc: 0.9471
[[2.0915950e-05 9.9997914e-01]
 [7.6505072e-05 9.9992347e-01]
 [9.9989688e-01 1.0305798e-04]
 [9.6844918e-01 3.1550799e-02]]
0.9316067341533995 0.9779366291438496
0.9936564745919789 0.9779364663974189
17080 21349
Model: "sequential_18"
______________________________

[[9.9988294e-01 1.1705901e-04]
 [9.9957424e-01 4.2580909e-04]
 [2.1844242e-09 1.0000000e+00]
 [3.2625267e-09 1.0000000e+00]]
0.948640621285072 0.9815612035159575
0.9943598575107713 0.9815612035159574
29890 34159
Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 13, 68, 128)       2176      
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 10, 65, 32)        65568     
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 7, 62, 8)          4104      
_________________________________________________________________
flatten_21 (Flatten)         (None, 3472)              0         
_________________________________________________________________
dense_61 (Dense)             (None, 1024)              3556352   
_______________________________________

In [ ]:
from scipy.stats import sem, t
from scipy import mean
confidence = 0.95
# data = [1, 2, 3, 4, 5]

naucList = len(aucListpositive1)
maucList = mean(aucListpositive1)
std_erraucList = sem(aucListpositive1)
haucList = std_erraucList * t.ppf((1 + confidence) / 2, naucList - 1)


nauprList = len(auprListpositive1)
mauprList = mean(auprListpositive1)
std_errauprList = sem(auprListpositive1)
hauprList = std_errauprList * t.ppf((1 + confidence) / 2, nauprList  - 1)



print("positive1: "+ str(maucList) + '  -+' + str(haucList), str(mauprList) + '  -+' + str(hauprList))

In [ ]:
mypd = []
mypd = pd.DataFrame(mypd)
mypd['AUC'],mypd['AUPR'] = aucListpositive1, auprListpositive1
mypd.to_csv('selectedModelresult(+1).csv')

In [ ]:
from scipy.stats import sem, t
from scipy import mean
confidence = 0.95
# data = [1, 2, 3, 4, 5]

naucList = len(aucListmines1)
maucList = mean(aucListmines1)
std_erraucList = sem(aucListmines1)
haucList = std_erraucList * t.ppf((1 + confidence) / 2, naucList - 1)


nauprList = len(auprListmines1)
mauprList = mean(auprListmines1)
std_errauprList = sem(auprListmines1)
hauprList = std_errauprList * t.ppf((1 + confidence) / 2, nauprList  - 1)



print("mines1: "+ str(maucList) + '  -+' + str(haucList), str(mauprList) + '  -+' + str(hauprList))

In [ ]:
mypd = []
mypd = pd.DataFrame(mypd)
mypd['AUC'],mypd['AUPR'] = aucListmines1, auprListmines1
mypd.to_csv('selectedModelresult(-1).csv')